# Notebook for data cleaning

In [1]:
import pandas as pd

In [10]:
df_main = pd.read_parquet("data/properties_main.parquet", engine="pyarrow", dtype_backend="numpy_nullable")
df_features = pd.read_parquet("data/properties_characteristics.parquet")


In [6]:
df_main

,property_code,property_reference,title,description,property_type,property_subtype,postal_code,address,locality,latitude,...,has_pantry,image_count,publisher_code,publisher_name,publisher_phone,price_per_sqm_sale,price_per_sqm_rent,total_monthly_cost,size_category,amenity_score
0,85001-L,85001-L,Apartamento Mobiliado para alugar com 1 quarto...,Apartamento Mobiliado para alugar com 1 quarto...,Apartamento,Padrão,14026591,"Jardim Nova Alianca, Ribeirao Preto","Jardim Nova Aliança,Ribeirão Preto,São Paulo,B...",-21.22099,...,False,16,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,49.019608,2500.0,medium,0
1,84999-S,84999-S,Cobertura Penthouse com 3 suites no bairro San...,Cobertura Penthouse com 3 suites no bairro San...,Apartamento,Cobertura,14020700,"Santa Cruz Do Jose Jacques, Ribeirao Preto","Santa Cruz Do José Jacques,Ribeirão Preto,São ...",<NA>,...,True,47,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,4934.210526,<NA>,<NA>,extra_large,6
2,84998-S,84998-S,Belissima Casa no Condominio do Jardim Nova Al...,Casa Sobrado a Venda – Residencial Nova Alianc...,Casa,Casa de Condomínio,14026551,"Jardim Nova Alianca, Ribeirao Preto","Jardim Nova Aliança,Ribeirão Preto,São Paulo,B...",-21.21515,...,False,22,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,5301.587302,<NA>,<NA>,extra_large,1
3,84996-L,84996-L,"Predio Comercial no Jardim Botanico, em Ribeir...","Predio Comercial no Jardim Botanico, em Ribeir...",Comercial,Prédio Inteiro,14021593,"Jardim Botanico, Ribeirao Preto","Jardim Botânico,Ribeirão Preto,São Paulo,Brasil",-21.21753,...,False,10,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,57.142857,16000.0,extra_large,0
4,84990-S,84990-S,Apartamento de 3 dormitorios na Quinta da Prim...,Encontre o imovel dos seus sonhos neste lindo ...,Apartamento,Padrão,14022100,"Quinta da Primavera, Ribeirao Preto","Quinta da Primavera,Ribeirão Preto,São Paulo,B...",-21.23351,...,False,36,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,8987.341772,<NA>,<NA>,medium,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11921,182-S,182-S,Terreno Padrão em São Carlos,", Área total 540,00 m²",Terreno,Terreno Padrão,13562060,"Planalto Paraiso, Sao Carlos","Planalto Paraíso,São Carlos,São Paulo,Brasil",-22.0076,...,False,4,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
11922,162-S,162-S,"Casa Padrao a venda no Jardim Bethania, Sao Ca...",Linda casa padrao a venda no bairro Jardim Bet...,Casa,Padrão,13561060,"Jardim Bethania, Sao Carlos","Jardim Bethânia,São Carlos,São Paulo,Brasil",-22.00854,...,False,3,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,2792.55625,<NA>,<NA>,large,0
11923,114-S,114-S,"Apartamento Padrao a Venda em Miguel Abdelnur,...",Conheca este lindo apartamento com 2 dormitori...,Apartamento,Padrão,13571385,"Distrito Industrial Miguel Abdelnur, Sao Carlos","Distrito Industrial Miguel Abdelnur,São Carlos...",-22.04302,...,False,15,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,5555.555556,<NA>,<NA>,small,0
11924,6-S,6-S,Terreno a venda no bairro Parque Santa Felicia...,Encontre o terreno dos seus sonhos no bairro P...,Terreno,Terreno Padrão,13563307,"Santa Felicia, Sao Carlos","Santa Felícia,São Carlos,São Paulo,Brasil",-21.99665,...,False,4,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0


In [21]:
# Droping columns not to be put in the model (id is kept to re-join if needed)
# Also selecting only properties from sao carlos
sao_carlos = df_main[df_main["locality"].apply(lambda row: row.split(",")[1] == "São Carlos")]
selected = (sao_carlos
.drop(columns=[
    "property_reference",
    "title",
    'description',
    'postal_code',
    'address',
    'latitude',
    'longitude', # A more complex model may use it to calculate distance to uptown, will not be used at first
    'locality', # Using neighborhoods may introduce too much sparsity in the model, will avoid at first
    'show_map',
    'has_sale_price',
    'has_rent_price', # Already visible by NaNs
    'image_count',
    'publisher_code',
    'publisher_name',
    'publisher_phone',
    'price_per_sqm_rent',
    'price_per_sqm_sale'
]))

In [22]:
# lets look at nan values
selected.isnull().sum().divide(len(selected))

property_code         0.000000
property_type         0.000000
property_subtype      0.000000
sale_price            0.227403
rent_price            0.772597
bedrooms              0.263636
bathrooms             0.184416
suites                0.610130
parking_spaces        0.305455
area_util             0.135065
area_total            0.020519
condominium_fee       0.575195
property_tax          0.394675
has_pool              0.000000
has_bbq               0.000000
has_gym               0.000000
has_playground        0.000000
has_sauna             0.000000
has_party_room        0.000000
has_sports_court      0.000000
has_24h_security      0.000000
has_laundry           0.000000
has_closet            0.000000
has_office            0.000000
has_pantry            0.000000
total_monthly_cost    0.772597
size_category         0.142208
amenity_score         0.000000
dtype: float64

.Para sale_price e rent_price, sabemos que os NaN correspondem ao status da propriedade (aluga-se/vende-se). Serao separados em 2 dataframes.
Para as outras colunas, sera analisado caso a caso para definir se os NaNs possuem significado ou sao faltas aleatorias de dados (MCAR).

In [25]:
# bedroom
print(sao_carlos["bedrooms"].unique())
sao_carlos[sao_carlos["bedrooms"].isnull()]

<IntegerArray>
[3, 1, 2, <NA>, 4, 5, 8, 15, 6, 7, 0, 10, 12, 11, 80]
Length: 15, dtype: Int64


,property_code,property_reference,title,description,property_type,property_subtype,postal_code,address,locality,latitude,...,has_pantry,image_count,publisher_code,publisher_name,publisher_phone,price_per_sqm_sale,price_per_sqm_rent,total_monthly_cost,size_category,amenity_score
23,84939-L,84939-L,Terreno comercial para alugar no bairro Santa ...,Excelente oportunidade de locacao de terreno c...,Comercial,Galpão/Depósito/Barracão,13563120,"Santa Felicia, Sao Carlos","Santa Felícia,São Carlos,São Paulo,Brasil",-21.99098,...,False,14,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,4.848485,1600.0,extra_large,0
27,84929-S,84929-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
28,84928-S,84928-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
29,84927-S,84927-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
32,84924-S,84924-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11916,445-L,445-L,Oportunidade imperdivel! Salao Comercial a ven...,Excelente imovel comercial localizado no bairr...,Comercial,Loja/Salão,13568715,"Douradinho, Sao Carlos","Douradinho,São Carlos,São Paulo,Brasil",-22.01742,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,5.053476,945.0,large,0
11917,416-L,416-L,Salao Comercial na Vila Monteiro (Gleba I) em ...,Salao Comercial na Vila Monteiro (Gleba I) em ...,Comercial,Loja/Salão,13560320,"Vila Monteiro (Gleba I), Sao Carlos","Vila Monteiro (Gleba I),São Carlos,São Paulo,B...",-22.01485,...,False,26,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,10.526316,1000.0,medium,0
11921,182-S,182-S,Terreno Padrão em São Carlos,", Área total 540,00 m²",Terreno,Terreno Padrão,13562060,"Planalto Paraiso, Sao Carlos","Planalto Paraíso,São Carlos,São Paulo,Brasil",-22.0076,...,False,4,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
11924,6-S,6-S,Terreno a venda no bairro Parque Santa Felicia...,Encontre o terreno dos seus sonhos no bairro P...,Terreno,Terreno Padrão,13563307,"Santa Felicia, Sao Carlos","Santa Felícia,São Carlos,São Paulo,Brasil",-21.99665,...,False,4,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0


Nota-se que NaN em 'bedrooms' equivalem a terrenos e salas comerciais. Nao havendo o valor 0 nessa coluna, logo eh possivel substituir nan por 0, porem tambem cabe decidir se essas propriedades serao inclusas no modelo. Eh seguro assumir que nas outras colunas relacionadas a estrutura, os NaN tambem correspondem a 0. acabei de ver e n eh assism entt fds

In [27]:
sao_carlos[sao_carlos["area_total"].isnull()]

,property_code,property_reference,title,description,property_type,property_subtype,postal_code,address,locality,latitude,...,has_pantry,image_count,publisher_code,publisher_name,publisher_phone,price_per_sqm_sale,price_per_sqm_rent,total_monthly_cost,size_category,amenity_score
27,84929-S,84929-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
28,84928-S,84928-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
29,84927-S,84927-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
32,84924-S,84924-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
33,84923-S,84923-S,Comercial Sala em São Carlos,Comercial Sala em São Carlos,Comercial,Conjunto Comercial/Sala,13561352,"Parque Faber Castell, Sao Carlos","Parque Faber Castell,São Carlos,São Paulo,Brasil",-22.01743,...,False,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11808,6127-L,6127-L,Apartamento Padrão em São Carlos,APARTAMENTO - 1/4 - JATIUCA - CENTRO,Apartamento,Padrão,13560235,"CENTRO, Sao Carlos","CENTRO,São Carlos,São Paulo,Brasil",-22.01671,...,True,18,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,2161.0,NaN,0
11831,5371-S,5371-S,"Casa a venda na Vila Marcelino, Sao Carlos - 3...",Excelente oportunidade! Casa padrao com 3 quar...,Casa,Padrão,13570594,"Vila Marcelino, Sao Carlos","Vila Marcelino,São Carlos,São Paulo,Brasil",-22.02759,...,False,20,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,4615.384615,<NA>,<NA>,medium,0
11836,5312-S,5312-S,Casa Padrão em São Carlos,"CASA - 2/4 - EUGENIO FRANCO DE CAMARGO, 2390 -...",Casa,Padrão,13569270,"Jardim Brasil, Sao Carlos","Jardim Brasil,São Carlos,São Paulo,Brasil",-22.01033,...,True,11,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,5957.446809,<NA>,<NA>,small,0
11857,5136-S,5136-S,Comercial Sala em São Carlos,"SALA - RUA ANTONIO CARLOS FERRAZ DE SALLES, 70...",Comercial,Conjunto Comercial/Sala,13563306,"Santa Felicia , Sao Carlos","Santa Felícia ,São Carlos,São Paulo,Brasil",-21.99617,...,False,6,2,Roca Administradora de Imóveis Eireli,(16) 3373-5000,<NA>,<NA>,<NA>,NaN,0


Observando area_util e area_total, terrenos e algumas salas comerciais nao possuem area_util, alem de faltas nao explicadas, para area_total, sao todas faltas de dados nao explcadas